In [ ]:
# ------------------------------------------------
import os, json, re, glob
from typing import Dict, Any, List, Tuple, Optional
import numpy as np
import torch
import pandas as pd

# Project imports
import sys
sys.path.append("/mnt/data")
from model_generation import GeneratedModel
from data_processing import get_dataset, combine_arrays, split_combined_data

# ---------- PyTorch 2.6 safe deserialization helpers ----------
try:
    from torch.serialization import add_safe_globals
    import torch.torch_version
    add_safe_globals([torch.torch_version.TorchVersion])
except Exception:
    pass

def safe_torch_load(path: str, map_location="cpu"):
    try:
        return torch.load(path, map_location=map_location)  # weights_only=True by default on 2.6
    except Exception as e1:
        print(f"⚠️ safe_torch_load: retrying with weights_only=False for {os.path.basename(path)} "
              f"(only do this if you trust the source). Error was: {e1}")
        return torch.load(path, map_location=map_location, weights_only=False)

# ---------- Defaults pulled from your search space ----------
DEFAULTS = {
    'num_conv_layers': 3,
    'filters_per_layer': 64,
    'kernel_size': 5,
    'stride': 1,
    'padding': 'same',
    'activation': 'relu',
    'batch_norm': True,
    'use_dropout': False,
    'dropout': 0.0,
    'pooling_type': 'none',
    'pool_size': 2,
    'residual_connections': False,
    'normalization': 'none',
    'initialization': 'default',
    'optimizer': 'adam',
    'learning_rate': 1e-3,
    'weight_decay': 0.0,
    'batch_size': 256,
}

REQUIRED_KEYS = [
    'num_conv_layers','filters_per_layer','kernel_size','stride','padding',
    'activation','batch_norm','use_dropout','dropout','pooling_type','pool_size',
    'residual_connections','normalization','initialization'
]

In [ ]:
def find_model_dirs(root: str) -> List[str]:
    """A model dir contains model_package.pth OR (config.json AND weights.pth)."""
    out = []
    for path in glob.glob(os.path.join(root, "*")):
        if not os.path.isdir(path):
            continue
        if (os.path.isfile(os.path.join(path, "model_package.pth")) or
            (os.path.isfile(os.path.join(path, "config.json")) and os.path.isfile(os.path.join(path, "weights.pth")))):
            out.append(path)
    return sorted(out)

def load_arch_and_state_and_ckpt(model_dir: str):
    """
    Returns (arch_config, state_dict, ckpt_path or None).
    Prefers model_package.pth; falls back to config.json + weights.pth.
    """
    pkg = os.path.join(model_dir, "model_package.pth")
    if os.path.isfile(pkg):
        pkg_data = safe_torch_load(pkg, map_location="cpu")
        arch_config = pkg_data.get("arch_config", {}) or {}
        state_dict  = pkg_data["state_dict"]
        # try to locate a copied original ckpt for later repair
        ckpts = glob.glob(os.path.join(model_dir, "*.ckpt"))
        ckpt_path = ckpts[0] if ckpts else None
        return arch_config, state_dict, ckpt_path

    cfg_path = os.path.join(model_dir, "config.json")
    wts_path = os.path.join(model_dir, "weights.pth")
    if not (os.path.isfile(cfg_path) and os.path.isfile(wts_path)):
        raise FileNotFoundError(f"missing model files in {model_dir}")
    with open(cfg_path, "r", encoding="utf-8") as f:
        cfg = json.load(f)
    arch_config = cfg.get("arch_config", {}) or {}
    state_dict = safe_torch_load(wts_path, map_location="cpu")
    ckpts = glob.glob(os.path.join(model_dir, "*.ckpt"))
    ckpt_path = ckpts[0] if ckpts else None
    return arch_config, state_dict, ckpt_path

def infer_from_state(state: Dict[str, torch.Tensor]) -> Dict[str, Any]:
    """Infer a few fields from state_dict (best-effort)."""
    # Count conv blocks
    num_layers = None
    pat = re.compile(r"^(?:model\.)?conv_blocks\.(\d+)\.")
    max_idx = -1
    for k in state.keys():
        m = pat.match(k)
        if m:
            max_idx = max(max_idx, int(m.group(1)))
    if max_idx >= 0:
        num_layers = max_idx + 1

    # Find a conv weight to read out_channels and kernel_size
    filters = None
    kernel = None
    for k, v in state.items():
        if k.endswith("weight") and isinstance(v, torch.Tensor) and v.ndim == 3:
            # Conv1d: [out_channels, in_channels, kernel_size]
            oc, ic, ks = v.shape
            filters = oc
            kernel = ks
            break

    # Residual connections: presence of projections.* is a hint
    residual = any(re.match(r"^(?:model\.)?projections\.\d+\.", k) for k in state.keys())

    return {
        "num_conv_layers": num_layers,
        "filters_per_layer": filters,
        "kernel_size": kernel,
        "residual_connections": residual,
    }

def try_arch_from_ckpt(ckpt_path: Optional[str]) -> Dict[str, Any]:
    """Try to recover hyper parameters from original Lightning .ckpt."""
    if not ckpt_path or not os.path.isfile(ckpt_path):
        return {}
    try:
        ckpt = safe_torch_load(ckpt_path, map_location="cpu")
        hp = ckpt.get("hyper_parameters") or ckpt.get("hparams") or {}
        # common patterns: either a nested dict or flat keys
        if isinstance(hp, dict):
            if "architecture_config" in hp and isinstance(hp["architecture_config"], dict):
                return hp["architecture_config"]
            # else try to pick expected keys from flat hp
            extracted = {k: hp[k] for k in REQUIRED_KEYS if k in hp}
            # handle alt names
            if "use_dropout" not in extracted and "dropout" in hp:
                extracted["use_dropout"] = float(hp["dropout"]) > 0.0
            return extracted
    except Exception as e:
        print(f"⚠️ could not read hyper_parameters from {os.path.basename(ckpt_path)}: {e}")
    return {}

def coerce_types(cfg: Dict[str, Any]) -> Dict[str, Any]:
    """Coerce to proper types expected by GeneratedModel."""
    out = dict(cfg)
    ints = {"num_conv_layers","filters_per_layer","kernel_size","stride","pool_size"}
    floats = {"dropout","learning_rate","weight_decay"}
    bools = {"batch_norm","use_dropout","residual_connections"}
    lowers = {"padding","activation","pooling_type","normalization","initialization","optimizer"}
    for k in list(out.keys()):
        v = out[k]
        try:
            if k in ints and v is not None:
                out[k] = int(v)
            elif k in floats and v is not None:
                out[k] = float(v)
            elif k in bools and v is not None:
                if isinstance(v, str):
                    out[k] = v.strip().lower() in ("true","1","yes")
                else:
                    out[k] = bool(v)
            elif k in lowers and v is not None:
                out[k] = str(v).strip().lower()
        except Exception:
            pass
    return out

def repair_arch_config(arch: Dict[str, Any], state: Dict[str, torch.Tensor], ckpt_path: Optional[str]) -> Dict[str, Any]:
    """Fill missing keys using ckpt hyperparameters, then state_dict inference, then defaults."""
    repaired = dict(arch or {})
    # 1) ckpt hyperparameters
    ckpt_arch = try_arch_from_ckpt(ckpt_path)
    for k, v in ckpt_arch.items():
        repaired.setdefault(k, v)
    # 2) state_dict inference (only for some keys)
    inferred = infer_from_state(state)
    for k, v in inferred.items():
        if v is not None:
            repaired.setdefault(k, v)
    # 3) defaults
    for k, v in DEFAULTS.items():
        repaired.setdefault(k, v)
    # types
    repaired = coerce_types(repaired)
    # make sure dropout flag is consistent
    if "use_dropout" in repaired and "dropout" in repaired:
        repaired["use_dropout"] = float(repaired["dropout"]) > 0.0 if repaired["dropout"] is not None else False
    return repaired

def clean_state_dict_for_model(state: Dict[str, torch.Tensor]) -> Dict[str, torch.Tensor]:
    out = {}
    for k, v in state.items():
        kk = k.split("model.", 1)[1] if k.startswith("model.") else k
        out[kk] = v.detach().cpu()
    return out

@torch.inference_mode()
def evaluate(model: torch.nn.Module, X: np.ndarray, y: np.ndarray, device: torch.device, batch_size: int = BATCH_SIZE):
    X_t = torch.as_tensor(X, dtype=torch.float32, device=device)
    preds = []
    for i in range(0, X_t.size(0), batch_size):
        xb = X_t[i:i+batch_size]
        preds.append(model(xb).detach().cpu().numpy())
    y_pred = np.vstack(preds).astype(np.float32)
    y_true = y.astype(np.float32)
    diff = y_pred - y_true
    mse = float(np.mean(diff**2))
    rmse = float(np.sqrt(mse))
    mae = float(np.mean(np.abs(diff)))
    return {"mse": mse, "rmse": rmse, "mae": mae}

# ---------- Collections and subset plan ----------
ALL_COLLECTIONS = [
    "equilatero_grande_garage",
    "equilatero_grande_outdoor",
    "equilatero_medio_garage",
    "equilatero_medio_outdoor",
    "isosceles_grande_indoor",
    "isosceles_grande_outdoor",
    "isosceles_medio_outdoor",
    "obtusangulo_grande_outdoor",
    "obtusangulo_pequeno_outdoor",
    "reto_grande_garage",
    "reto_grande_indoor",
    "reto_grande_outdoor",
    "reto_medio_garage",
    "reto_medio_outdoor",
    "reto_n_quadrado_grande_indoor",
    "reto_n_quadrado_grande_outdoor",
    "reto_n_quadrado_pequeno_outdoor",
    "reto_pequeno_garage",
    "reto_pequeno_outdoor",
]

def group_by_location(collections: List[str], locations: List[str]) -> List[str]:
    return [name for name in collections if any(loc in name for loc in locations)]

SUBSET_PLAN = {
    "garage":  group_by_location(ALL_COLLECTIONS, ["garage"]),
    "outdoor": group_by_location(ALL_COLLECTIONS, ["outdoor"]),
    "indoor":  group_by_location(ALL_COLLECTIONS, ["indoor"]),
}

def load_subset_data_cached(db_name: str):
    """Load garage/outdoor/indoor once and cache."""
    cache = {}
    for subset, colls in SUBSET_PLAN.items():
        arrays = [get_dataset(name, db_name) for name in colls]
        combined = combine_arrays(arrays)
        X, y = split_combined_data(combined)
        cache[subset] = (X, y)
    return cache

In [ ]:
# =====================================================================
# CONFIGURATION (edit these variables as needed)
# =====================================================================
MODELS_ROOT = "/home/admindi/sbenites/WirelessLocation/validation/model_per_dataset_validation/full_models"
DB_NAME = "wifi_fingerprinting_data"
OUT_CSV = "export_models_validation_long.csv"
OUT_PIVOT_CSV = "export_models_validation_pivot.csv"
METRICS = "rmse"   # choose from "rmse", "mse", "mae"
LIMIT = 0          # 0 = evaluate all models, >0 = limit number
BATCH_SIZE = 4096
# =====================================================================


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)

model_dirs = find_model_dirs(MODELS_ROOT)
print(f"Found {len(model_dirs)} model folders in {MODELS_ROOT}")

# Load data once
subset_cache = load_subset_data_cached(DB_NAME)
input_size, output_size = subset_cache["garage"][0].shape[1], subset_cache["garage"][1].shape[1]
print(f"Input/Output dims: {input_size} -> {output_size}")

rows = []
for mdir in model_dirs:
    model_name = os.path.basename(mdir.rstrip(os.sep))
    try:
        arch_raw, state, ckpt_path = load_arch_and_state_and_ckpt(mdir)
        arch = repair_arch_config(arch_raw, state, ckpt_path)
        if any(k not in arch for k in REQUIRED_KEYS):
            missing = [k for k in REQUIRED_KEYS if k not in arch]
            print(f"[{model_name}] ⚠️ architecture still missing {missing}; using defaults may degrade fidelity.")
        model = GeneratedModel(input_size=input_size, output_size=output_size, architecture_config=arch)
        cleaned = clean_state_dict_for_model(state)
        missing, unexpected = model.load_state_dict(cleaned, strict=False)
        if missing:
            print(f"[{model_name}] missing keys: {missing[:6]}{'...' if len(missing)>6 else ''}")
        if unexpected:
            print(f"[{model_name}] unexpected keys: {unexpected[:6]}{'...' if len(unexpected)>6 else ''}")
        model.to(device).eval()
    except Exception as e:
        print(f"[{model_name}] ❌ load failed after repair: {e}")
        continue

    for subset in ("garage","outdoor","indoor"):
        try:
            X, Y = subset_cache[subset]
            m = evaluate(model, X, Y, device=device)
            rows.append({"model": model_name, "subset": subset, **m})
            print(f"[{model_name}] ✅ {subset}: RMSE={m['rmse']:.4f} (MSE={m['mse']:.4f}, MAE={m['mae']:.4f})")
        except Exception as e:
            print(f"[{model_name}] ⚠️ eval {subset} failed: {e}")

# Display tables (no CSVs)
df = pd.DataFrame(rows).sort_values(["model", "subset"])
display(df[["model", "subset", "rmse", "mse", "mae"]])

pivot = df.pivot_table(index="model", columns="subset", values=METRIC_TO_PIVOT, aggfunc="min").reset_index()
display(pivot)

Device: cuda
Found 105 model folders in /home/admindi/sbenites/WirelessLocation/validation/model_per_dataset_validation/full_models
Input/Output dims: 3 -> 2
[all_data_run0_depth7_model5] ✅ garage: RMSE=0.2954 (MSE=0.0872, MAE=0.2540)
[all_data_run0_depth7_model5] ✅ outdoor: RMSE=0.2981 (MSE=0.0889, MAE=0.2552)
[all_data_run0_depth7_model5] ✅ indoor: RMSE=0.3897 (MSE=0.1519, MAE=0.3212)
[all_data_run0_depth8_model5] ❌ load failed: 'num_conv_layers'
[all_data_run1_depth5_model1] ✅ garage: RMSE=0.2954 (MSE=0.0873, MAE=0.2540)
[all_data_run1_depth5_model1] ✅ outdoor: RMSE=0.2981 (MSE=0.0889, MAE=0.2552)
[all_data_run1_depth5_model1] ✅ indoor: RMSE=0.3897 (MSE=0.1519, MAE=0.3211)
[all_data_run1_depth5_model7] ❌ load failed: 'num_conv_layers'
[all_data_run2_depth3_model0] ✅ garage: RMSE=0.2954 (MSE=0.0873, MAE=0.2540)
[all_data_run2_depth3_model0] ✅ outdoor: RMSE=0.2981 (MSE=0.0889, MAE=0.2552)
[all_data_run2_depth3_model0] ✅ indoor: RMSE=0.3897 (MSE=0.1519, MAE=0.3212)
[all_data_run2_depth

NameError: name 'METRICS_TO_SHOW' is not defined